# Summary

Notebook to run the embedding process for multiple documents

In [ ]:


from embed_documents_3 import EmbedDocuments

## Test

In [ ]:
# Set paths
input_text_path = ("/Users/stephengodfrey/OneDrive - numanticsolutions.com"
                   "/Engagements/Projects/ccc_policy_assistant/data/crawls/")
embeddings_path = ("/Users/stephengodfrey/OneDrive - numanticsolutions.com"
                   "/Engagements/Projects/ccc_policy_assistant/data/embeddings-vai")
collection_name = "crawl_docs1"
gcs_bucket_name = "ccc-chromadb-vai"

embed_loader = EmbedDocuments(input_text_path=input_text_path,
                              embeddings_path=embeddings_path,
                              collection_name=collection_name,
                              gcs_bucket_name=gcs_bucket_name)

# Read input files
embed_loader.get_input_filenames()

# Read input files
embed_loader.read_text_data()

# Chunk text
embed_loader.chunk_input_text()

# Embed
# embed_loader.embed()

# Copy Embeddings to GCS
# embed_loader.copy_embeddings_to_gcs()


## Test embeddings

In [ ]:
embed_loader.input_df
mask = embed_loader.input_df["url"].str.find("MiraCosta")>=0
embed_loader.input_df[mask]


In [15]:
embed_loader.docs


for doc in embed_loader.docs:
    if doc.metadata == {'url': 'https://en.wikipedia.org/wiki/MiraCosta_College'}:
        print(doc)
        if doc.page_content.find("Center of Excellence in bioprocessing ")>=0:
            print(doc.page_content)

page_content='MiraCosta College is a public community college serving coastal Northern San Diego County in California with two campuses, one in eastern Oceanside and the other in Cardiff-by-the-Sea, and many satellite locations, including the Community Learning Center in western Oceanside. MiraCosta was founded in 1934. Classes began in a wing at Oceanside High School. Originally, MiraCosta was known as Oceanside-Carlsbad Junior College. There were 122 students enrolled and 20 faculty members. It was the second community college to be established in San Diego County. Only 16 courses were initially offered, and they counted as credit toward advanced standing at the University of California. [citation needed] By 1959, enrollment in Oceanside-Carlsbad Junior College had grown to 500 full-time students in classrooms on 30 acres (12 ha) adjacent to Oceanside High School. In 1960, voters approved a separate community college district and a bond issue to provide funding for a new college camp